In [1]:
# train a dominant detector
from pygod.detector import DOMINANT

from pygod.utils import load_data

data = load_data('weibo') # in PyG format
model = DOMINANT(num_layers=4, epoch=20)  # hyperparameters can be set here
model.fit(data)  # input data is a PyG data object

# get outlier scores on the training data (transductive setting)
score = model.decision_score_
print("training score: ", score)
# predict labels and scores on the testing data (inductive setting)
pred, score = model.predict(data, return_score=True)

print("predict score: ", score)

/home/shark/miniforge3/envs/gad_test_c/lib/python3.12/site-packages/pygod/utils/utility.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)
/h

training score:  tensor([1740.8672,  917.8134,  341.4017,  ..., 1194.1462,  622.9437,
        2881.2761])
predict score:  tensor([1489.1364,  813.9673,  298.5081,  ..., 1055.1726,  553.7695,
        2564.6584])


In [3]:
# Import necessary libraries
from pygod.detector import CONAD
from pygod.utils import load_data
from sklearn.metrics import roc_auc_score

# Load data in PyG format
data = load_data('weibo')

# Initialize and train the DOMINANT model
model = CONAD()  # Hyperparameters can be set here
model.fit(data)  # Input data is a PyG data object

# Get outlier scores on the training data (transductive setting)
train_score = model.decision_score_
print("Training score: ", train_score)

# Predict labels and scores on the testing data (inductive setting)
pred, test_score = model.predict(data, return_score=True)
print("Predict score: ", test_score)

# Extract ground truth labels from the data object
y_true = data.y.numpy()  # Ground truth labels (1 for anomalies, 0 for normal)

# Compute AUC using sklearn's roc_auc_score
auc_score = roc_auc_score(y_true, test_score)
print(f"AUC Score: {auc_score:.4f}")

/home/shark/miniforge3/envs/gad_test_c/lib/python3.12/site-packages/pygod/utils/utility.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)
/h

Training score:  tensor([168.8004,  77.7198,  59.4289,  ..., 100.1070, 103.9973, 351.9234])
Predict score:  tensor([167.3499,  77.0197,  59.2666,  ...,  99.1355, 103.0802, 347.6328])
AUC Score: 0.8801


In [6]:
import torch
x = torch.eye(4) + 2
x, x.sum(1).pow(-0.5)

(tensor([[3., 2., 2., 2.],
         [2., 3., 2., 2.],
         [2., 2., 3., 2.],
         [2., 2., 2., 3.]]),
 tensor([0.3333, 0.3333, 0.3333, 0.3333]))

In [5]:
from torch_geometric.utils import add_self_loops, add_remaining_self_loops
import torch

edge_index = torch.tensor([[0, 1, 2, 1], [1, 2, 0, 1]], dtype=torch.long)
edge_index, _ = add_remaining_self_loops(edge_index, num_nodes=3)

print(edge_index)
# 输出:
# tensor([[0, 1, 2, 0, 1, 2],
#         [1, 2, 0, 0, 1, 2]])

tensor([[0, 1, 2, 0, 1, 2],
        [1, 2, 0, 0, 1, 2]])


In [14]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.empty(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        # Step 6: Apply a final bias vector.
        out = out + self.bias

        return out

    def message(self, x_j, norm, x_i):
        # x_j has shape [E, out_channels]
        print("x_i: ", x_i.shape)
        print("x_j: ", x_j.shape)
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

In [ ]:
# 1. 创建一个随机图
# 假设我们有一个包含 4 个节点的图，每个节点有 16 维特征
num_nodes = 5
num_features = 8

# 节点特征矩阵 x: [num_nodes, num_features]
x = torch.rand((num_nodes, num_features))  # 随机生成节点特征

# 边索引 edge_index: [2, num_edges]
# 这里我们手动定义一些边 (0-1, 1-2, 2-3, 3-0)
edge_index = torch.tensor([[0, 1, 2, 3], [1, 2, 3, 0]], dtype=torch.long)

# 2. 定义 GCNConv 层
in_channels = num_features  # 输入特征维度
out_channels = 16           # 输出特征维度
conv = GCNConv(in_channels, out_channels)

# 3. 前向传播
output = conv(x, edge_index)

# 打印结果
print("输入节点特征形状:", x.shape)
print("输出节点特征形状:", output.shape)
print("输出节点特征:")
print(output)

x_i:  torch.Size([8, 16])
x_j:  torch.Size([8, 16])
输入节点特征形状: torch.Size([4, 8])
输出节点特征形状: torch.Size([4, 16])
输出节点特征:
tensor([[-0.1596,  0.3181, -0.1366,  0.1896,  0.4349, -0.0353, -0.2308, -0.2075,
         -0.1839, -0.4057,  0.0528,  0.1265,  0.3195, -0.2710,  0.2117, -0.4142],
        [-0.2343,  0.2402,  0.0196,  0.1361,  0.2385,  0.0307, -0.1567, -0.1859,
         -0.1602, -0.4438,  0.0430, -0.0325,  0.2829, -0.2290,  0.0760, -0.2730],
        [-0.3008,  0.1754, -0.1129,  0.1463,  0.3277,  0.1270, -0.2049, -0.2250,
         -0.2286, -0.4420,  0.0071, -0.1554,  0.1823, -0.2443,  0.3046, -0.3444],
        [-0.2260,  0.2534, -0.2691,  0.1997,  0.5242,  0.0611, -0.2791, -0.2466,
         -0.2523, -0.4039,  0.0169,  0.0037,  0.2189, -0.2863,  0.4403, -0.4856]],
       grad_fn=<AddBackward0>)
